In [8]:


#///////// Implementación de dos métodos de agrupamiento usando evolución diferencial /////////

import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans2
from numpy.linalg import *
import pylab
from sklearn import datasets
from math import *

print('>>>>>  Algoritmo Evolución Diferencial  <<<<<<<')
print('\n------ Datos de Entrada ------\n')
print('database=Iris')
print('Poblacion = 100')
print('d=4')
print('K=2')

np.random.seed(1)

#---------------VERIABLES----------------

iris = datasets.load_iris() #se carga el conjunto de datos de IRIS

#---se seleccionan los primeros 100 datos
data = iris.data[:100]
target = iris.target[:100]
data = (data - data.min(axis=0))/(data.max(axis=0)-data.min(axis=0))
N = len(data)
n = len(data[0]) #longitud del individuo
K = 2 #numero de clusters
chrom = []#vector que guarda los K cromosomas de (K + K*d)

#--------------------FUNCIONES------------------
def dist_to_centroids(CHROM, IND, clusters_activos):
    distancias = np.array([])
    
    #print('IND: ', IND)
    #print('umbral: ',UMBRALES)
    
    
    #print('clusters activos: ', clusters_activos)
    for activo in clusters_activos:         
        cluster = []
        for n in range(K + activo*len(IND), K + activo*len(IND) + len(IND)):
            cluster.append(CHROM[n])
        #print('cluster: ', cluster)
        acum = 0.0
        for p in range(len(IND)):#se recorre cada elemento del individuo
            acum += pow(IND[p] - cluster[p], 2)
        #print('acum: ',acum)
        distancias = np.append(distancias, sqrt(acum))
    
    return distancias

#funcion para encontrar los clusters activos del cromosoma
def get_active_clusters(CHROM):
    active = np.zeros(K) # se inicializan los valores de los umbrales en ceros(inactivos)
    
    #se recorren las primeras K posiciones del cromosoma (umbrales de activacion)
    inactive = []#almacena los clusters inactivos
    for umbral in range(K):
        if CHROM[umbral] > 0.5:
            active[umbral] = 1
        else:
            inactive.append(umbral)

    #se valida que al menos dos umbrales esten activos
    if sum(active) < 2.0:
        while(sum(active) < 2.0):            
            r = np.random.randint(len(inactive)) 
            i = inactive[r]
            CHROM[i] = np.random.randint(500,1000)*.001
            #print('inactive:', inactive, '  irand:', i)
            active[ i ] = 1
            inactive.pop(r)
    
    return [i for i,x in enumerate(active) if x==1.0]


def get_chromosome(UMBRALES, CENTROIDS):
    #se unen los umbrales con los centroides
    for centroid in CENTROIDS: UMBRALES = np.concatenate((UMBRALES, data_training[centroid]))
    
    return UMBRALES
    
        
    
print ('rand: ', np.random.randint(3))

#funcion objetivo 1
#funcion objetivo 2

#se seleccionan 100 indices desordenados entre [0,100]
idx = np.random.permutation(N)
data_training = data[idx[:int(N*.8)]]
target_training = target[idx[:int(N*.8)]]


#se seleccionan los K centroides de manera aleatoria
centroids = np.random.permutation(int(N*.8))[:K]
print('\n------ centroides: ------\n')
print(centroids)


#se generan los valores Ti,j (umbrales de activacion del centroide)
chrom = np.random.rand(K)
#se inicializan el cromosoma de dimension Kmax + Kmax*d
chrom = get_chromosome(chrom, centroids)
    
print('\n------ Cromosoma: ------\n')
print(chrom,'\n')
print('\n------ umbrales activos: ------\n')
clusters_activos = get_active_clusters(chrom)
print(clusters_activos)

#dis = distancia(data_training[0], umbrales)
#print(dis)


>>>>>  Algoritmo Evolución Diferencial  <<<<<<<

------ Datos de Entrada ------

database=Iris
Poblacion = 100
d=4
K=3
rand:  1

------ centroides: ------

[16 62]

------ Cromosoma: ------

[0.53883106 0.55282198 0.48148148 0.41666667 0.85365854 0.82352941
 0.11111111 0.45833333 0.12195122 0.05882353] 


------ umbrales activos: ------

[0, 1]


In [60]:
#def init_clusters():
    #ind = len(data_training) / K #cantidad de puntos por cluster
    #for
    
#DB-Index
def DB_Index(DATA, CLUSTERS):
    
    R = []
    
    for idCluster in range(K):
        
        S = []    
        for cluster in CLUSTERS:
            Nc = len(cluster)
            #print('Cluster:',cluster)
            acum = 0.0
            for X in cluster:
                #print(X[1])
                acum += abs(X[1])*abs(X[1])
            S.append(sqrt((1.0/Nc)*acum))
            #print('Aqui')

        d = []
        for i in range(K):
            if i != idCluster:
                acum = 0.0
                for j in range(len(data_training[0])):
                    Ci = DATA[centroids[idCluster]]
                    Cj = DATA[centroids[i]]
                    acum += pow(Ci[j] - Cj[j], 2)
                d.append(sqrt(acum))
            else:
                d.append([])
        
        #print('S:',S)
        #print('d:',d)
        
        VECTOR = []
        for i in range(K):
            if i != idCluster:
                VECTOR.append( (S[idCluster] + S[i]) / d[i] )
        
        #print('VECTOR:',VECTOR)
        R.append(max(VECTOR))
        #print('\n\n')
        
    #print('\n\n')
    #print('R:', R)
    
    return (1.0/K)*sum(R)

#CS-Measure
def CS_Measure():
    return 0


def get_clusters(DATA, CHROM):
    CLUSTERS = [] #almacena los individuos de cada uno de los clusters
    for c in range(K): CLUSTERS.append([])
        
    clusters_activos = get_active_clusters(CHROM)
    #print('activos: ',clusters_activos)
        
    #para cada individuo calcular la distancia a todos los clusters activos
    for IND in DATA:
        #se obtienen las distancias de individuo actual hacia los clusters activos
        all_dist = dist_to_centroids(CHROM, IND, clusters_activos)
        #se obtiene el indice del cluster con la distancia mas cercana
        best = clusters_activos[ list(all_dist).index(min(all_dist)) ]
        #if best == 0: print('Dist:', all_dist,'  best:', best)
        CLUSTERS[best].append([list(IND), min(all_dist)])
    
    return CLUSTERS

def f1(DATA, CLUSTERS):
    return 1.0/(DB_Index(DATA,CLUSTERS) + exp(1))




t = 0
lb = -1.5
ub = 2.0
CR = 0.8
F = 0.5
N = len(data_training)
u = np.zeros((N,n))
Tmax = 50

DATA_CLUSTERS = []

while (t < Tmax):  
    
    #se generan los valores Ti,j (umbrales de activacion del centroide)
    chrom = np.random.rand(K)
    #se inicializan el cromosoma de dimension Kmax + Kmax*d
    chrom = get_chromosome(chrom, centroids)
    
    DATA_CLUSTERS = get_clusters(data_training, chrom)
    
    for i in range(N):
        [r1, r2, r3] = np.random.permutation(N)[:3]
        irand = np.random.randint(n)
        for j in range(n):
            if np.random.random() < CR or j == irand:
                u[i,j] = data_training[r3,j] + F * (data_training[r1,j] - data_training[r2,j])
                if u[i,j] < lb or u[i,j] > ub:
                    u[i,j] = (data_training[r1,j] + data_training[r2,j] + data_training[r3,j])/3.0
            else:
                u[i,j] = data_training[i,j]
    
    DATA_CLUSTERS2 = get_clusters(u, chrom)
    
    #print('CLUSTERS:\n', (DATA_CLUSTERS[1][0]))
    #print('CLUSTERS:\n', (DATA_CLUSTERS2[1][0]))
    
    f1x = f1(data_training, DATA_CLUSTERS)
    #print('DB: ',f1x)
    f1u = f1(u, DATA_CLUSTERS2)
    #print('DB2: ',f1u)
    
    #print('f1x:',f1x, '  f1u:',f1u)
    
    if(f1u > f1x):
        data_training = u.copy()
        print('fit: ', f1u)
    else:
        #se seleccionan los K centroides de manera aleatoria
        centroids = np.random.permutation(int(N*.8))[:K]
        
        
    
        
        
    #break
    t+=1

    

fit:  0.27162139339015445
fit:  0.2679601543593448
fit:  0.2552421731584942
fit:  0.25561466960040663
fit:  0.2464820575515423
fit:  0.23500130218309506
fit:  0.2525347262976783
fit:  0.23809562200124015
fit:  0.21391915935504724
fit:  0.24394707735155896
fit:  0.26085131162597186
fit:  0.2757836851055452
fit:  0.21534905062285653
fit:  0.22264538872951115
fit:  0.24683336497348354
fit:  0.25933192945919215
fit:  0.25476170171050017
fit:  0.22599416240600434
fit:  0.2453107129534359
fit:  0.236143694025701
fit:  0.23981057121707444
fit:  0.24893447945017144
fit:  0.2515326822773542
